# Creating ASN File for splittag Products

In [1]:
from costools import timefilter

import calcos 

from astropy.io import fits                                            
import numpy as np
import glob
import os

The following tasks in the costools package can be run with TEAL:
         splittag                 timefilter                 x1dcorr


### timefilter example from https://costools.readthedocs.io/en/latest/timefilter.html

# get the data

In [3]:
from astroquery.mast import Observations

In [32]:
# Observations.query_criteria(obsid)

In [33]:
# nuv_downloads = Observations.download_products(Observations.get_product_list(Observations.query_criteria(obs_id = 'lbbd01020')), # GET the NUV data on WD1057+719 with the G230L grating; only the _x1dsum and _asn
#                                            download_dir='.' , extension='fits', mrp_only=True, cache=False)

In [48]:
query = Observations.query_criteria(obs_id = 'LBEK02010')
pl = Observations.get_product_list(query)
corrtag_mask = (pl["productSubGroupDescription"] == "CORRTAG_A") | (pl["productSubGroupDescription"] == "CORRTAG_B")
Observations.download_products(pl[corrtag_mask][:2])

Local Path,Status,Message,URL
str53,str8,object,object
./mastDownload/HST/lbek02niq/lbek02niq_corrtag_a.fits,COMPLETE,None,None
./mastDownload/HST/lbek02niq/lbek02niq_corrtag_b.fits,COMPLETE,None,None


In [50]:
timefilter.TimelineFilter("lbek02niq_corrtag_a.fits", "temp_corrtag_a.fits",  # your input names will be different
                              "sun_alt > 0.")
timefilter.TimelineFilter("lbek02niq_corrtag_b.fits", "temp_corrtag_b.fits",
                              "sun_alt > 0.")
# repeat this for all exposures

### Creating an ASN file out of the above files

In [51]:
# this properly assigns the type of exposure in a way that CalCOS will recognize
type_dict = {'WAVECAL' : 'EXP-AWAVE',
             'EXTERNAL/SCI' : 'EXP-FP'}

files = glob.glob('temp_corrtag_a*') ## This will be different depending on your output name ## only need the A segment in the ASN, and calcos will find B for you

for f in files:
    
    # Adding the file details to the association table
    rootnames = [f]  # MEMNAME
    types = [type_dict[fits.getval(f, 'EXPTYPE')]] # MEMTYPE
    included = [True] # MEMPRSNT
    
    # Adding the ASN details to the end of the association table
    # the rootname needs to be the full name, not just the rootname
    asn_root = os.path.basename(f.split('corrtag')[0][:-1]) # removing the extra "_" at the end of this
    rootnames.append(asn_root.upper()) # MEMNAME
    types.append('PROD-FP') # MEMTYPE
    included.append(True) # MEMPRSNT
    
    # Putting together the fits table
    #   40 is the number of characters allowed in this field. If your rootname is longer than 40, 
    #     you will need to increase this
    c1 = fits.Column(name='MEMNAME', array=np.array(rootnames), format='40A') 
    c2 = fits.Column(name='MEMTYPE', array=np.array(types), format='14A')
    c3 = fits.Column(name='MEMPRSNT', format='L', array=included)
    t = fits.BinTableHDU.from_columns([c1, c2, c3])
    
    # Writing the fits table
    t.writeto(asn_root.lower()+'_asn.fits')

    print('Saved: {}'.format(asn_root.lower()+'_asn.fits'))


Saved: temp_asn.fits


In [52]:
asnfile = 'temp_asn.fits'
print(fits.getdata(asnfile))

[('temp_corrtag_a.fits', 'EXP-FP',  1) ('TEMP', 'PROD-FP',  1)]


### Then run CalCOS on those individual files

In [53]:
for f in glob.glob('temp_asn.fits'):
    calcos.calcos(f, outdir='calibrated_filtered')

CALCOS version 3.3.9
numpy version 1.19.2
astropy version 4.0.2
Begin 28-Oct-2020 18:06:17 EDT
Association file = temp_asn.fits

TIME-TAG calibration -- 28-Oct-2020 18:06:20 EDT
Input     temp_corrtag_a.fits
OutTag    calibrated_filtered/temp_corrtag_a.fits
OutFlt    calibrated_filtered/temp_flt_a.fits
OutCounts calibrated_filtered/temp_counts_a.fits
OutFlash  calibrated_filtered/temp_lampflash_a.fits
DETECTOR  FUV, segment A
EXPTYPE   EXTERNAL/SCI
OPT_ELEM  G160M, CENWAVE 1577, FPOFFSET -2
APERTURE  PSA

BADTCORR  OMIT
RANDCORR  OMIT (already complete)
TEMPCORR  OMIT (already complete)
GEOCORR   OMIT (already complete)
DGEOCORR  OMIT
DEADCORR  OMIT (already complete)
    exptime has been corrected to 400.032
PHACORR   OMIT (already complete)
DOPPCORR  OMIT (already complete)
XTRACTAB= lref$x1v17414l_1dx.fits
DISPTAB = lref$05i1639ml_disp.fits
BRFTAB  = lref$x1u1459il_brf.fits
FLATCORR  OMIT (already complete)
WAVECORR  OMIT (already complete)
BRSTCORR  OMIT
TRCECORR  OMIT
ALGNCORR  OM

FUVA spectrum was found at y = 525.32 vs. nominal y = 480.79
    error estimate for y location = 999.00, FWHM = -1.00
Spectrum will be extracted at y = 480.79

TIME-TAG calibration -- 28-Oct-2020 18:06:53 EDT
Input     temp_corrtag_b.fits
OutTag    calibrated_filtered/temp_corrtag_b.fits
OutFlt    calibrated_filtered/temp_flt_b.fits
OutCounts calibrated_filtered/temp_counts_b.fits
OutFlash  calibrated_filtered/temp_lampflash_b.fits
DETECTOR  FUV, segment B
EXPTYPE   EXTERNAL/SCI
OPT_ELEM  G160M, CENWAVE 1577, FPOFFSET -2
APERTURE  PSA

BADTCORR  OMIT
RANDCORR  OMIT (already complete)
TEMPCORR  OMIT (already complete)
GEOCORR   OMIT (already complete)
DGEOCORR  OMIT
DEADCORR  OMIT (already complete)
    exptime has been corrected to 400.032
PHACORR   OMIT (already complete)
DOPPCORR  OMIT (already complete)
XTRACTAB= lref$x1v17414l_1dx.fits
DISPTAB = lref$05i1639ml_disp.fits
BRFTAB  = lref$x1u1459il_brf.fits
FLATCORR  OMIT (already complete)
WAVECORR  OMIT (already complete)
BRSTCORR  O

In [54]:
ls

DayNight.html             lbek02niq_corrtag_a.fits  temp_asn.fits
DayNight.ipynb            lbek02niq_corrtag_b.fits  temp_corrtag_a.fits
calibrated_filtered/      mastDownload/             temp_corrtag_b.fits


In [58]:
from astropy.io import fits
from astropy.table import Table

In [59]:
Table.read('./calibrated_filtered/temp_x1dsum.fits')

SEGMENT,EXPTIME,NELEM,WAVELENGTH [16384],FLUX [16384],ERROR [16384],GROSS [16384],GCOUNTS [16384],NET [16384],BACKGROUND [16384],DQ [16384],DQ_WGT [16384],BACKGROUND_PER_PIXEL [16384]
,s,,Angstrom,erg / (Angstrom cm2 s),erg / (Angstrom cm2 s),ct / s,ct,ct / s,ct / s,,,ct / (pix s)
bytes4,float64,int32,float64,float32,float32,float32,float32,float32,float32,int16,float32,float32
FUVA,400.032,16384,1562.7002363643853 .. 1763.2022630432666,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,128 .. 128,0.0 .. 0.0,0.0 .. 0.0
FUVB,400.032,16384,1374.5475567326278 .. 1574.9566125479453,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,128 .. 128,0.0 .. 0.0,0.0 .. 0.0
